# LLM4AD Tutorial -- Use LLM4AD to Solve TSP Problem

In [ ]:
!pip install llm4ad

## Installation
1. Create a Python environment with Python 3.9-3.11.
    ```shell
    conda create -n llm4ad310 python=3.10
    ```
2. Activate the `llm4ad310` environment.
    ```shell
    conda activate llm4ad310
    ```
3. Install `llm4ad` and other required packages used in this tutorial.
    ```shell
    pip install llm4ad torch tensorboard jupyter notebook
    ```


## Step 1: Template program
We need to create a **template program** for an algorithm that includes the following:
1) All the **Python packages** required for designing the algorithm.
2) The **names of the functions**, their **arguments**, and the **type hints** of their return values.
3) **Descriptive information** for each argument provided in the function’s docstring.
4) A **simple implementation** for the function (for the `EoH` algorithm, you don’t have to include a full implementation—a simple pass statement with a return is fine).

This template program serves as a "boilerplate" for the search methods. The parameter descriptions in the docstring can also help an LLM better understand the algorithm’s purpose, which can reduce errors during the search process.

Please note that the destination node is also added to the arguments list to provide more information.

In [ ]:
template_program = '''
import numpy as np

def select_next_node(current_node: int, destination_node: int, unvisited_nodes: np.ndarray, distance_matrix: np.ndarray) -> int: 
    """Design an algorithm to select the next node in each step.
    Args:
        current_node    : ID of the current node.
        destination_node: ID of the destination node.
        unvisited_nodes : Array of IDs of unvisited nodes.
        distance_matrix : Distance matrix of nodes.
    Return:
        ID of the next node to visit.
    """
    next_node = unvisited_nodes[0]
    return next_node
'''

## Step 2: Evaluation interface
We should provide an evaluator for LLM4AD that outputs a score for an algorithm, which is higher the better. And always return `None` if the result is illegal.

For the convenience, we have provided functions to produce datasets and calculate the distance for an algorithm. The usages for these functions are shown below.
- Import `GetData` to get TSP instances.
- Import `cal_avg_distance` for score calculation.

In [ ]:
from evaluation import cal_avg_distance
from get_instance import GetData

import numpy as np

def select_next_node(current_node: int, destination_node: int, 
                     unvisited_nodes: np.ndarray, distance_matrix: np.ndarray) -> int: 
    next_node = unvisited_nodes[0]  # visit the first node in unvisited list
    return next_node

data = GetData(n_instance=1, n_cities=50).generate_instances()  # we create one TSP instance, the instance has 50 cities
distance = cal_avg_distance(instance_data=data, n_instance=1, n_cities=50, evaluated_algorithm=select_next_node)
print(f'distance: {distance}')

In [ ]:
from evaluation import cal_avg_distance
from get_instance import GetData

import numpy as np

def select_next_node(current_node: int, destination_node: int, 
                     unvisited_nodes: np.ndarray, distance_matrix: np.ndarray) -> int: 
    next_node = unvisited_nodes[-1]  # visit the last node in unvisited list
    return next_node

data = GetData(n_instance=1, n_cities=50).generate_instances()  # we create one TSP instance, the instance has 50 cities
distance = cal_avg_distance(instance_data=data, n_instance=1, n_cities=50, evaluated_algorithm=select_next_node)
print(f'distance: {distance}')

In LLM4AD platform, we should complete following steps:
1. Extend the `llm4ad.base.evaluate.Evaluation` class.
2. Passing evaluate arguments to the super class if necessary.
    - Passing the `template_program` to the super class. The `template_program` here acts as a biolerplate for all evaluated algorithms. 
    - Passing the `task_description` parameter to the super class to  
    - If `use_numba_accelerate` is set to `True`, the numba wrapper will be added to the evaluated algorithm automatically. It should be noted that the acceleration provided by numba is not promising in all cases.
    - Set a `timeout_seconds` to restrict the maximum evaluation time of an algorithm. If the evaluation violate the time restriction, the evaluation process will be terminated automatically and the score of the evaluated algorihtm will be set to `None`.
3. override the `evaluate_program` function and return the score of the evaluated algorithm.
    - The `program_str` argument is the string type of the algorithm to be evaluated. The string is provided for the requirements for penalizing the length of the algorithm code, or compile the program in an alternative way.
    - The `callable_func` is the compiled version for the evlauted algorithm. It is callable and thus can be directly used for evaluation.

In our TSP task, we invoke `avg_dist` function to get the average distance over all TSP instances (test cases). **Please note that the distance has been inverted. As the distance is used as the score of an algorithm, which is higher the better.**

In [ ]:
from llm4ad.base import LLM, Evaluation, SecureEvaluator
from evaluation import cal_avg_distance
from get_instance import GetData


class TSPEval(Evaluation):
    """Evaluator for traveling salesman problem."""

    def __init__(self):
        super().__init__(
            template_program=template_program, 
            task_description='',
            use_numba_accelerate=False,
            timeout_seconds=20
        )
        getData = GetData(n_instance=16, n_cities=50)
        self._datasets = getData.generate_instances()

    def evaluate_program(self, program_str: str, callable_func: callable):
        dist = cal_avg_distance(self._datasets, 16, 50, callable_func)
        return -dist

**Tips:** Here is a tip for debugging: We first create an evaluator and evaluate the `template_program` in debug mode.

In [ ]:
tsp_eval = TSPEval()
evaluator = SecureEvaluator(tsp_eval, debug_mode=True)
score, time = evaluator.evaluate_program_record_time(template_program)
print(f'score: {score} eval time: {time}')

## Step 3: LLM interface
The LLM interface defines how we access to the LLM. The following cell shows an example for using an HTTPS API to query remote LLMs. 

**Tips:** Invoke `draw_sample` function to test the accessibility.

In [ ]:
from llm4ad.tools.llm.llm_api_https import HttpsApi

llm = HttpsApi(
    host='api.bltcy.ai',
    key='sk-dHvAGxNBt11jP9QqdOJse7YjA59h5BHTSS2zWaJZT9e2Ltws',
    model='gpt-4o-mini'
)

response = llm.draw_sample('Hello!')
response

'Hi there! Nice to meet you. How can I help today?\n\nI can assist with:\n- Explaining concepts or answering questions\n- Writing and editing (emails, essays, resumes, etc.)\n- Coding and debugging\n- Math, science, or homework help\n- Brainstorming ideas or planning (projects, trips, events)\n- Translation or summarization\n- General chat or guidance\n\nTell me what you’re aiming for or drop a question, and I’ll tailor my help.'

## Step 4: Automated algorithm design
Install following packages since we will show the usage of tensorboard profiler later.

In [ ]:
!pip install numpy==1.26.0
!pip install torch
!pip install tensorboard
!pip install tensorboardX
!rm -rf TSP

1. Create a profiler (for recording and visualizing the results).
2. Initialize a search method.
3. Run.

You can visualize the results through tensorboard:
```shell
tensorboard --logdir TSP --port 6006
```
And the results will be demonstrated in http://127.0.0.1:6006.

In [14]:
from llm4ad.tools.profiler import TensorboardProfiler
from llm4ad.method.eoh import EoH

profiler = TensorboardProfiler(log_dir='TSP')

eoh = EoH(llm=llm, 
          evaluation=tsp_eval, 
          profiler=profiler, 
          max_sample_nums=20,
          num_samplers=1, 
          num_evaluators=1)

eoh.run()

LLM Parameters
--------------------------------------------------------------------
  - LLM: HttpsApi
  - do_auto_trim: True
  - debug_mode: False
  - _host: api.bltcy.ai
  - _key: sk-dHvAGxNBt11jP9QqdOJse7YjA59h5BHTSS2zWaJZT9e2Ltws
  - _model: gpt-5-nano
  - _timeout: 60
  - _kwargs: {}
  - _cumulative_error: 0
Problem Parameters
--------------------------------------------------------------------
  - Problem: TSPEval
  - task_description: 
  - use_numba_accelerate: False
  - use_protected_div: False
  - protected_div_delta: 1e-05
  - random_seed: None
  - timeout_seconds: 20
  - exec_code: True
  - safe_evaluate: True
  - daemon_eval_process: False
  - fork_proc: auto
Method Parameters
--------------------------------------------------------------------
  - Method: EoH
  - _max_generations: 10
  - _max_sample_nums: 20
  - _pop_size: 5
  - _selection_num: 2
  - _use_e2_operator: True
  - _use_m1_operator: True
  - _use_m2_operator: True
  - _num_samplers: 1
  - _num_evaluators: 1
  - 

In [ ]:
!tensorboard --logdir TSP